In [2]:
#------------- Importation des modules -----------------


import pyaudio   #Importation du module PyAudio (émettre le son)
import wave      #Importation du module wave (lire, telecharger ou jouer un fichier Wav)
import numpy as np #Importation du module numpy (taleau à deux dimensions, PI, fonction maths (sin, cos, etc...))
import matplotlib.pyplot as plt #Importation du module matplotlib.pyplot (Graphique)
from os.path import abspath   #Importation du module
import time  #Importation du module(créer des pauses dans le programme)
import soundfile as sf      #Importation du module soundfile permettant de traiter et lire des fichiers audio
import sounddevice as sd    #Importation du module sounddevice qui permet de  lire et d'enregistrer des tableaux NumPy contenant des signaux audio
import scipy.io.wavfile   #Importation du module scipy.io.wavfile
from scipy.io.wavfile import write #Importation de la fonction write de scipy.io.wavfile qui permet d'insérer les données d'un fichier WAV dans un tableau
import scipy.signal as sig   #Importation du module scipy.signal qui permet d'importer des fonctions de filtrage
from scipy.signal import butter, filtfilt, lfilter
from functools import partial


# --------- Création de la fonction décodage --------


def xor(a, b):  
   
    result = []  # Initialisation du résultat
   

    # si les bits sont idem, alors XOR vaut 0, sinon 1
    for i in range(1, len(b)): 
        if a[i] == b[i]: 
            result.append('0') 
        else: 
            result.append('1') 
   
    return ''.join(result) 


def mod2div(divident, divisor): 
   
    pick = len(divisor) 
    tmp = divident[0 : pick] 
   
    while pick < len(divident): 
   
        if tmp[0] == '1': 

            tmp = xor(divisor, tmp) + divident[pick] 
   
        else:    
            tmp = xor('0'*pick, tmp) + divident[pick] 
    
        pick += 1
    
    if tmp[0] == '1': 
        tmp = xor(divisor, tmp) 
    else: 
        tmp = xor('0'*pick, tmp) 
   
    checkword = tmp 
    return checkword 

def decodeData(Message_decode, Clé): 
    l_Clé = len(Clé) 
    appended_data = str(Message_decode) + '0'*(l_Clé-1) 
    remainder = mod2div(appended_data, Clé) 
   
    return remainder 


# --------- Création du filtre passe-bande ----------


def butter_highpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='high', analog=False)
    return b, a

def butter_highpass_filter(data, cutoff, fs, order=5): #Définition d'une fonction passe-bande
    b, a = butter_highpass(cutoff, fs, order=order)
    y = filtfilt(b, a, data, padlen=0)
    return y


# -------------Reception ---------------


CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 2
fs = 47000
RATE = 47000
Fe = 47000
RECORD_SECONDS = 7
Clé = '10011'

WAVE_OUTPUT_FILENAME = "Son_recu.wav"

p = pyaudio.PyAudio()
 
stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)
 
print("* Enregistrement...")
 
frames = []
 
for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)): #Enregistrement du message
    data = stream.read(CHUNK) #Enregistrement du son arrivant dans le microphone
    frames.append(data) #Tableau frames prend les valeurs enregistrés
 
print("* Enregistré")
 
stream.stop_stream()
stream.close()
p.terminate()
 
 
wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb') #Sauvegarde du message enregistré
wf.setnchannels(CHANNELS)
wf.setsampwidth(p.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b''.join(frames))
wf.close()


signal, Fe = sf.read(r'Son_recu.wav') #La variable signal prend en valeur le son enregistré



# -------------- Filtrage --------------------
  

time.sleep(0.5)

with open("Son_recu.wav", "rb") as file_in:
    with open("Binaire.txt", "w") as file_out:
        for byte in iter(partial(file_in.read, 1), b''):
            file_out.write(format(ord(byte), '0>8b'))
            
cutoff = 20500
order = 6

y = butter_highpass_filter(signal, cutoff, fs, order)

plt.figure (figsize = (12,5))
plt.grid(True)
plt.plot(y)
plt.xlabel('Temps [s]')
plt.ylabel("Amplitude $x(t)$")
plt.title('Signal filtré(t)')
plt.show()

Nbits = len(y)    #Nombre de bits initial (taille du message entré en binaire)
baud = 100   #Débit de transmission en bit/seconde
Ns = int(Fe/baud)        #Nombre de symboles par bit (Fréq d'echan / Débit binaire) 
N = int(Nbits * Ns)   #Nombre de bits totals


# ------------ Démodulation -----------------3


N = len(y)   #N prend la valeur du nombre de valeurs présentes dans la liste y qui est le signal filtré
T = np.arange(0.0, 2*(N/2))/Fe   #Vecteur temps des porteuses de démodulation


Fp1 = 23000     #Fréquence de la porteuse 1
Fp2 = 21000     #Fréquence de la porteuse 2

A1 = 1   #Amplitude de la porteuse 1
A2 = 1   #Amplitude de la porteuse 2

S1 = A1*np.sin(2*np.pi*Fp1*T) #Utilisation de la même formule des porteuses que pour envoyer le message
S2 = A2*np.sin(2*np.pi*Fp2*T) #Utilisation de la même formule des porteuses que pour envoyer le message

bit1 = y[:, 1]*S1  #La variable bit1 prend en valeur le produit de la deuxième colonne de y et S1
bit0 = y[:, 1]*S2  #La variable bit1 prend en valeur le produit de la deuxième colonne de y et S2
   
y1 = [] #Création d'un tableau y1
y2 = [] #Création d'un tableau y2

z=0
for z in range(0, 2*N, Ns):
    y1.append(np.trapz(bit1[z:z+Ns],T[z:z+Ns]))
    y2.append(np.trapz(bit0[z:z+Ns],T[z:z+Ns]))
    
    
comparateur = (np.array(y1) > np.array(y2))


# ------------ Définition de la fonction décodage -------------


data = str(signal)

def encodeData(data, Clé): 
    l_Clé = len(Clé) 

    appended_data = data + '0'*(l_Clé-1) 
    
    remainder = mod2div(appended_data, Clé) 
   

    codeword = data + remainder 
    return codeword

# ------------ Liste pour stocker le message démodulé -----------------

data_crc= encodeData(data,Clé)

Message_démodule = []  #Création d'un tableau 

for ii in range (0, len(comparateur)):
    
    if comparateur [ii] == False:   #Si la case du tableau est négative alors la dernière case du tableau prend 0
        Message_démodule.extend([int(0)])
        
    if comparateur [ii] == True:
         Message_démodule.extend([int(1)]) #Si la case du tableau est positive alors la dernière case du tableau prend 1
        
             
print("Le message binaire démodulé est:\n ", Message_démodule)


# -------------- Décodage du message -----------------


Message_decode = [] #Création d'un tableau

for w in range (0, len(Message_démodule), 2): #De 0 jusque la fin de la liste "Message_démodule" avec un pas de 2 car on utilise un décodage synchrone Manchester
    Message_decode.extend(str(Message_démodule[w]))
    
    
Message_decode = list(map(int, Message_decode))  
check = decodeData(str(Message_decode), Clé) #La variable check prend la valeur de


for valeur in range(0, len(Message_decode), 8):
    print("Le reste de la division après décodage est -> " + check) 
    temp = "0" * (len(Clé) - 1) 


# -------- Conversion binaire ASCII ---------


bin_data = ""  #Création d'une variable pouvant contenir des caractères
for elem in Message_decode:   #Pour chaque élement dans le tableau message_decode
    bin_data += str(elem)  #Bin_data prend la valeur du tableau en string
    
data_reçu =' ' #Création d'une variable pouvant contenir des caractères

def BinaryToDecimal(binary):  #Création d'une fonction pouvant convertir le mesasge binaire en décimal
    binary1 = binary #La variable binary1 prend la valeur binary
    decimal, i, n = 0, 0, 0 #initialisation des variables
    
    while(binary != 0): #Tant que binary n'est pas égal à 0
        dec = binary % 10  #La variable dec prend binary % 10
        decimal = decimal + dec * pow(2, i) 
        binary = binary//10
        i += 1
    return (decimal) #Retourne la valeur finale

for i in range(0, len(bin_data), 8): 
    temp_data = int(bin_data[i+1:i+8]) #Utilisation de temp_data pour obtenir la valeur décimale correspondante de (temp_data)
      
    decimal_data = BinaryToDecimal(temp_data) #La variable decimal_data prend la BinaryToDecimal valeur de temp_data après être passé dans la fonction

    

    # Décodage de la valeur décimale renvoyée par la fonction BinarytoDecimal en utilisant char(decimal_data)
    data_reçu = data_reçu + chr(decimal_data) 


print("Le message reçu est :", data_reçu) #Affichage du message reçu, démodulé, décodé et reconverti en ASCII.   

OSError: [Errno -9998] Invalid number of channels